## Load Australia 5 minute dispatch price

In [ ]:
import pandas as pd
import numpy as np 

# Load original file
file_path = 'data/australia_data/AEMO_PUBLIC_DVD_DISPATCHPRICE_wide.zip'
df = pd.read_csv(file_path)

## See if all days are complete

In [24]:

# group by calendar day
daily = (
    df.groupby(['year', 'month', 'day'])['fivemin']
      .agg(min_fivemin='min',      # should be 1
           max_fivemin='max',      # should be 288
           intervals='count')      # should be 288
      .reset_index()
)

# any day that isn’t perfect
problem_days = daily.query('intervals != 288 or min_fivemin != 1 or max_fivemin != 288')

print(problem_days.head())
print('Total problem days:', len(problem_days))


     year  month  day  min_fivemin  max_fivemin  intervals
0    2009      7    1            1          287        287
31   2009      8    1            1          287        287
62   2009      9    1            1          287        287
92   2009     10    1            1          287        287
123  2009     11    1            1          287        287
Total problem days: 116


## Solve days that are not complete

In [32]:
df.head()
print(df_raw.columns.tolist())


['year', 'month', 'day', 'fivemin', 'rrp_nsw', 'rrp_qld', 'rrp_sa', 'rrp_tas', 'rrp_vic']


In [33]:
# keep a copy of the original file with the time-keeping columns
df_raw = df.copy()       # ← has year, month, day, fivemin

FULL_SET = np.arange(1, 289, dtype=np.int16)

def gaps(group):
    return sorted(set(FULL_SET).difference(group['fivemin']))

daily = (df_raw
         .groupby(['year','month','day'])
         .agg(min_fivemin=('fivemin','min'),
              max_fivemin=('fivemin','max'),
              intervals  =('fivemin','count'))
         .reset_index())

problem_days = daily.query('intervals != 288')
print(problem_days.head())


     year  month  day  min_fivemin  max_fivemin  intervals
0    2009      7    1            1          287        287
31   2009      8    1            1          287        287
62   2009      9    1            1          287        287
92   2009     10    1            1          287        287
123  2009     11    1            1          287        287


In [35]:

# ------------------------------------------------------------
# 0. ensure we are starting from the wide DataFrame ----------
# (keep a copy so later cells that melt / clean can reuse it)
df_wide = df_raw.copy()       # ← df_raw is whatever you read from the ZIPs

# ------------------------------------------------------------
# 1. make datetime the index, sorted -------------------------
df_wide = df_wide.sort_values(['year', 'month', 'day', 'fivemin'])
df_wide['datetime'] = pd.to_datetime(
        df_wide['year'].astype(str)  + '-' +
        df_wide['month'].astype(str).str.zfill(2) + '-' +
        df_wide['day'].astype(str).str.zfill(2)   + ' ' +
        ((df_wide['fivemin']-1)//12).astype(str).str.zfill(2) + ':' +   # hour
        (((df_wide['fivemin']-1)%12)*5).astype(str).str.zfill(2) +      # minute
        ':00'
)
df_wide = df_wide.set_index('datetime')
df_wide.head()

,year,month,day,fivemin,rrp_nsw,rrp_qld,rrp_sa,rrp_tas,rrp_vic
datetime,,,,,,,,,
2009-07-01 00:00:00,2009,7,1,1,16.941263,17.650000,16.730280,15.671540,15.50000
2009-07-01 00:05:00,2009,7,1,2,17.709524,18.810089,17.820490,16.057039,15.50000
2009-07-01 00:10:00,2009,7,1,3,17.678644,18.617599,18.123159,15.902460,15.39000
2009-07-01 00:15:00,2009,7,1,4,16.736212,18.611300,17.623659,14.273130,12.81297
2009-07-01 00:20:00,2009,7,1,5,15.638840,17.650000,16.334089,13.241490,11.80000


In [37]:

# ------------------------------------------------------------
# 2. build the full 5-minute grid ----------------------------
start = df_wide.index.min().normalize()                         # 00:00 first day
end   = df_wide.index.max().normalize() + pd.Timedelta('1D') - pd.Timedelta('5min')
full_idx = pd.date_range(start, end, freq='5T', tz=df_wide.index.tz)

df_wide = df_wide.reindex(full_idx)    # missing rows → NaN
df_wide.head()

,year,month,day,fivemin,rrp_nsw,rrp_qld,rrp_sa,rrp_tas,rrp_vic
2009-07-01 00:00:00,2009.0,7.0,1.0,1.0,16.941263,17.650000,16.730280,15.671540,15.50000
2009-07-01 00:05:00,2009.0,7.0,1.0,2.0,17.709524,18.810089,17.820490,16.057039,15.50000
2009-07-01 00:10:00,2009.0,7.0,1.0,3.0,17.678644,18.617599,18.123159,15.902460,15.39000
2009-07-01 00:15:00,2009.0,7.0,1.0,4.0,16.736212,18.611300,17.623659,14.273130,12.81297
2009-07-01 00:20:00,2009.0,7.0,1.0,5.0,15.638840,17.650000,16.334089,13.241490,11.80000


In [38]:

# ------------------------------------------------------------
# 3. interpolate only *single-step* gaps ---------------------
price_cols = [c for c in df_wide.columns if c.startswith('rrp_')]

df_wide[price_cols] = (
    df_wide[price_cols]
      .interpolate(method='linear', limit=1, limit_direction='both')
)

# ------------------------------------------------------------
# sanity: every day must now have 288 rows -------------------
assert (df_wide.groupby(df_wide.index.normalize()).size() == 288).all()
print("✓ All days complete and interpolated")
df_wide.head()

✓ All days complete and interpolated


,year,month,day,fivemin,rrp_nsw,rrp_qld,rrp_sa,rrp_tas,rrp_vic
2009-07-01 00:00:00,2009.0,7.0,1.0,1.0,16.941263,17.650000,16.730280,15.671540,15.50000
2009-07-01 00:05:00,2009.0,7.0,1.0,2.0,17.709524,18.810089,17.820490,16.057039,15.50000
2009-07-01 00:10:00,2009.0,7.0,1.0,3.0,17.678644,18.617599,18.123159,15.902460,15.39000
2009-07-01 00:15:00,2009.0,7.0,1.0,4.0,16.736212,18.611300,17.623659,14.273130,12.81297
2009-07-01 00:20:00,2009.0,7.0,1.0,5.0,15.638840,17.650000,16.334089,13.241490,11.80000


In [43]:
# keep a copy of the original file with the time-keeping columns
df_raw = df_wide.copy()       # ← has year, month, day, fivemin

FULL_SET = np.arange(1, 289, dtype=np.int16)

def gaps(group):
    return sorted(set(FULL_SET).difference(group['fivemin']))

daily = (df_raw
         .groupby(['year','month','day'])
         .agg(min_fivemin=('fivemin','min'),
              max_fivemin=('fivemin','max'),
              intervals  =('fivemin','count'))
         .reset_index())

problem_days = daily.query('intervals != 288')
print(problem_days.head())
len(problem_days)


Empty DataFrame
Columns: [year, month, day, min_fivemin, max_fivemin, intervals]
Index: []


0

In [45]:
# df_wide is already re-indexed and interpolated
idx = df_wide.index          # shortcut

df_wide['year']  = idx.year
df_wide['month'] = idx.month
df_wide['day']   = idx.day
df_wide['fivemin'] = ((idx - idx.normalize()).total_seconds() // 300 + 1
                      ).astype('int16')     # 1 … 288

# now repeat the diagnostic
daily = (df_wide
         .groupby(['year','month','day'])['fivemin']
         .agg(intervals='count',
              min_fivemin='min',
              max_fivemin='max')
         .reset_index())

print(daily['intervals'].value_counts())   # → 288    n_days
print(daily.query('intervals != 288'))     # empty
df_wide.head()


intervals
288    3530
Name: count, dtype: int64
Empty DataFrame
Columns: [year, month, day, intervals, min_fivemin, max_fivemin]
Index: []


,year,month,day,fivemin,rrp_nsw,rrp_qld,rrp_sa,rrp_tas,rrp_vic
2009-07-01 00:00:00,2009,7,1,1,16.941263,17.650000,16.730280,15.671540,15.50000
2009-07-01 00:05:00,2009,7,1,2,17.709524,18.810089,17.820490,16.057039,15.50000
2009-07-01 00:10:00,2009,7,1,3,17.678644,18.617599,18.123159,15.902460,15.39000
2009-07-01 00:15:00,2009,7,1,4,16.736212,18.611300,17.623659,14.273130,12.81297
2009-07-01 00:20:00,2009,7,1,5,15.638840,17.650000,16.334089,13.241490,11.80000


In [47]:
df = df_wide.copy()

# Compute minutes since midnight
df['minutes_since_midnight'] = (df['fivemin'] - 1) * 5

# Now create full timestamp
df['datetime'] = pd.to_datetime(df['year'].astype(str) + '-' +
                                df['month'].astype(str).str.zfill(2) + '-' +
                                df['day'].astype(str).str.zfill(2) + ' ' +
                                (df['minutes_since_midnight'] // 60).astype(str).str.zfill(2) + ':' +
                                (df['minutes_since_midnight'] % 60).astype(str).str.zfill(2) + ':00',
                                format='%Y-%m-%d %H:%M:%S')

df.head()


,year,month,day,fivemin,rrp_nsw,rrp_qld,rrp_sa,rrp_tas,rrp_vic,minutes_since_midnight,datetime
2009-07-01 00:00:00,2009,7,1,1,16.941263,17.650000,16.730280,15.671540,15.50000,0,2009-07-01 00:00:00
2009-07-01 00:05:00,2009,7,1,2,17.709524,18.810089,17.820490,16.057039,15.50000,5,2009-07-01 00:05:00
2009-07-01 00:10:00,2009,7,1,3,17.678644,18.617599,18.123159,15.902460,15.39000,10,2009-07-01 00:10:00
2009-07-01 00:15:00,2009,7,1,4,16.736212,18.611300,17.623659,14.273130,12.81297,15,2009-07-01 00:15:00
2009-07-01 00:20:00,2009,7,1,5,15.638840,17.650000,16.334089,13.241490,11.80000,20,2009-07-01 00:20:00


In [48]:

df['MTU (CET/CEST)'] = df['datetime'].dt.strftime('%d/%m/%Y %H:%M:%S') + ' - ' + \
    (df['datetime'] + pd.Timedelta(minutes=5)).dt.strftime('%d/%m/%Y %H:%M:%S')
df.head()

,year,month,day,fivemin,rrp_nsw,rrp_qld,rrp_sa,rrp_tas,rrp_vic,minutes_since_midnight,datetime,MTU (CET/CEST)
2009-07-01 00:00:00,2009,7,1,1,16.941263,17.650000,16.730280,15.671540,15.50000,0,2009-07-01 00:00:00,01/07/2009 00:00:00 - 01/07/2009 00:05:00
2009-07-01 00:05:00,2009,7,1,2,17.709524,18.810089,17.820490,16.057039,15.50000,5,2009-07-01 00:05:00,01/07/2009 00:05:00 - 01/07/2009 00:10:00
2009-07-01 00:10:00,2009,7,1,3,17.678644,18.617599,18.123159,15.902460,15.39000,10,2009-07-01 00:10:00,01/07/2009 00:10:00 - 01/07/2009 00:15:00
2009-07-01 00:15:00,2009,7,1,4,16.736212,18.611300,17.623659,14.273130,12.81297,15,2009-07-01 00:15:00,01/07/2009 00:15:00 - 01/07/2009 00:20:00
2009-07-01 00:20:00,2009,7,1,5,15.638840,17.650000,16.334089,13.241490,11.80000,20,2009-07-01 00:20:00,01/07/2009 00:20:00 - 01/07/2009 00:25:00


In [49]:

# Now "melt" the price columns → create one row per area
df_melted = df.melt(id_vars=['MTU (CET/CEST)'],
                    value_vars=['rrp_nsw', 'rrp_qld', 'rrp_sa', 'rrp_tas', 'rrp_vic'],
                    var_name='Area',
                    value_name='Day-ahead Price (EUR/MWh)')
df_melted.head()

,MTU (CET/CEST),Area,Day-ahead Price (EUR/MWh)
0,01/07/2009 00:00:00 - 01/07/2009 00:05:00,rrp_nsw,16.941263
1,01/07/2009 00:05:00 - 01/07/2009 00:10:00,rrp_nsw,17.709524
2,01/07/2009 00:10:00 - 01/07/2009 00:15:00,rrp_nsw,17.678644
3,01/07/2009 00:15:00 - 01/07/2009 00:20:00,rrp_nsw,16.736212
4,01/07/2009 00:20:00 - 01/07/2009 00:25:00,rrp_nsw,15.638840


In [50]:

# Clean 'Area' column → remove 'rrp_' prefix
df_melted['Area'] = df_melted['Area'].str.replace('rrp_', '', regex=False)
df_melted.head()

,MTU (CET/CEST),Area,Day-ahead Price (EUR/MWh)
0,01/07/2009 00:00:00 - 01/07/2009 00:05:00,nsw,16.941263
1,01/07/2009 00:05:00 - 01/07/2009 00:10:00,nsw,17.709524
2,01/07/2009 00:10:00 - 01/07/2009 00:15:00,nsw,17.678644
3,01/07/2009 00:15:00 - 01/07/2009 00:20:00,nsw,16.736212
4,01/07/2009 00:20:00 - 01/07/2009 00:25:00,nsw,15.638840


In [51]:
# Add 'Sequence' column
df_melted['Sequence'] = 'Without sequence'

# Final column order
df_target = df_melted[['MTU (CET/CEST)', 'Area', 'Sequence', 'Day-ahead Price (EUR/MWh)']]

# Inspect result
print(df_target.head(10))
print(df_target.tail(10))
print(df_target.columns.tolist())

                              MTU (CET/CEST) Area          Sequence  \
0  01/07/2009 00:00:00 - 01/07/2009 00:05:00  nsw  Without sequence   
1  01/07/2009 00:05:00 - 01/07/2009 00:10:00  nsw  Without sequence   
2  01/07/2009 00:10:00 - 01/07/2009 00:15:00  nsw  Without sequence   
3  01/07/2009 00:15:00 - 01/07/2009 00:20:00  nsw  Without sequence   
4  01/07/2009 00:20:00 - 01/07/2009 00:25:00  nsw  Without sequence   
5  01/07/2009 00:25:00 - 01/07/2009 00:30:00  nsw  Without sequence   
6  01/07/2009 00:30:00 - 01/07/2009 00:35:00  nsw  Without sequence   
7  01/07/2009 00:35:00 - 01/07/2009 00:40:00  nsw  Without sequence   
8  01/07/2009 00:40:00 - 01/07/2009 00:45:00  nsw  Without sequence   
9  01/07/2009 00:45:00 - 01/07/2009 00:50:00  nsw  Without sequence   

   Day-ahead Price (EUR/MWh)  
0                  16.941263  
1                  17.709524  
2                  17.678644  
3                  16.736212  
4                  15.638840  
5                  13.728767  
6

In [52]:
output_path = 'parquet_files/australia_data.parquet'
df_target.to_parquet(output_path, index=False)

### Check

In [53]:
df = df_target.copy()

# Extract the date part (first 10 characters of the string)
df['date'] = df['MTU (CET/CEST)'].str[:10]


interval_counts = df.groupby(['date', 'Area']).size().reset_index(name='num_intervals')


# How many unique interval counts exist?
print(interval_counts['num_intervals'].value_counts())

# Optional — show days that don't have 288 intervals
non_288_days = interval_counts[interval_counts['num_intervals'] != 288]
print(non_288_days['date'].unique())
print(interval_counts.head())


num_intervals
288    17650
Name: count, dtype: int64
[]
         date Area  num_intervals
0  01/01/2010  nsw            288
1  01/01/2010  qld            288
2  01/01/2010   sa            288
3  01/01/2010  tas            288
4  01/01/2010  vic            288


In [55]:
nas = df[df['MTU (CET/CEST)'].isna()]
nas

,MTU (CET/CEST),Area,Sequence,Day-ahead Price (EUR/MWh),date
